In [1]:
from train import main
import torch

model, train_loader, test_loader = main(dict(resume="1umk4zt7", save_path="models/dummy.pt"), use_wandb=False, num_epochs=0)

100%|██████████| 1/1 [00:13<00:00, 13.58s/it]
